In [17]:
import pandas as pd
df_1 = pd.read_csv('model_cost.csv')
df_1.head()

,Model,cost,No of tokens,no of calls,cost in USD (before discount)
0,Codegen,0.0018,10000,1,0.018
1,Codegen,0.0018,10000,100,1.800
2,Codegen,0.0018,10000,500,9.000
3,Codegen,0.0018,10000,1000,18.000
4,Codegen,0.0018,10000,10000,180.000


In [18]:
Q1 = "What is the unit cost for model Codegen?"  
Q2 = "which model is most expensive"  
Q3 = "how much ZenCode will cost for 20000 no of calls"  
Q4 = "create a line chart showing the cost of the 'MyCode' model based on the number of calls and cost"  
Q5 = "Summarize the data in 100 words."  
Q6 = 'List down the models for EMEA?'  
Q7 = 'Which is the most expensive model in EMEA?'  

In [19]:
json_data_1 = df_1.to_json(orient='records')
json_data_1

'[{"Model":"Codegen","cost":0.0018,"No of tokens":10000,"no of calls":1,"cost in USD (before discount)":0.018},{"Model":"Codegen","cost":0.0018,"No of tokens":10000,"no of calls":100,"cost in USD (before discount)":1.8},{"Model":"Codegen","cost":0.0018,"No of tokens":10000,"no of calls":500,"cost in USD (before discount)":9.0},{"Model":"Codegen","cost":0.0018,"No of tokens":10000,"no of calls":1000,"cost in USD (before discount)":18.0},{"Model":"Codegen","cost":0.0018,"No of tokens":10000,"no of calls":10000,"cost in USD (before discount)":180.0},{"Model":"ZenCode","cost":0.0006,"No of tokens":10000,"no of calls":1,"cost in USD (before discount)":0.006},{"Model":"ZenCode","cost":0.0006,"No of tokens":10000,"no of calls":100,"cost in USD (before discount)":0.6},{"Model":"ZenCode","cost":0.0006,"No of tokens":10000,"no of calls":500,"cost in USD (before discount)":3.0},{"Model":"ZenCode","cost":0.0006,"No of tokens":10000,"no of calls":1000,"cost in USD (before discount)":6.0},{"Model":"

In [20]:
prompt_json = """Here is the input dataframe in JSON format:
{json_data}
Please answer the question based on the input dataframe. It provides model cost information.
 Don't generate SQL code.
 Don't import a new data source.
 Use the correct dataframe name.
 Use the exact column names as they appear in the dataframe without changing the case.
 Return the answer in JSON format, including the Python Pandas code used to derive the answer.
 
Example Output Format:
{{
  "code": "df[['Model', 'cost']].sort_values(by='cost', ascending=False).head(1)",
  "answer": "The model with the highest cost is 'Codegen'."
}}

Think step by step before answering.
Here is the query: {Q}
"""


In [21]:
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

In [22]:
project_id = "a15722af-476f-4ad2-ad25-090750dd14e2"  # WatonX project ID

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="Nt8BVcy9pRD8bDngmalyAujN02hPV7PsuJu6bpUr6zxN"
)

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MAX_NEW_TOKENS: 8000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5
}
watsonx_llm_code = WatsonxLLM(model_id="ibm/granite-34b-code-instruct", 
                              url=credentials.url,
                              apikey=credentials.api_key,
                              project_id=project_id,
                              params=parameters)

/Users/durgaprasad/Desktop/_WEEKEND/DAY4-AI/env3_11/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-34b-code-instruct' is in deprecated state from 2025-04-16 until 2025-07-17. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [23]:
response = watsonx_llm_code.generate([prompt_json.format(json_data=json_data_1, Q=Q1)])

generated_text = response.generations[0][0].text

print(response)


generations=[[Generation(generation_info={'finish_reason': 'eos_token'}, text="Here is the code to answer the query:\n```python\ndf[df['Model'] == 'Codegen']['cost'].mean()\n```\nThe unit cost for model Codegen is 0.0018.")]] llm_output={'token_usage': {'generated_token_count': 48, 'input_token_count': 896}, 'model_id': 'ibm/granite-34b-code-instruct', 'deployment_id': None} run=[RunInfo(run_id=UUID('13897c95-2f8d-4d24-ab96-7e4a0d6e13c7'))] type='LLMResult'


In [24]:
parts = generated_text.split("```python")
code = parts[1].split("```")[0].strip() if len(parts) > 1 else None
answer = parts[1].split("```")[1].strip() if len(parts) > 1 else None

print("Code:", code)
print("Answer:", answer)

Code: df[df['Model'] == 'Codegen']['cost'].mean()
Answer: The unit cost for model Codegen is 0.0018.


In [25]:
df= df_1
df.head()

,Model,cost,No of tokens,no of calls,cost in USD (before discount)
0,Codegen,0.0018,10000,1,0.018
1,Codegen,0.0018,10000,100,1.800
2,Codegen,0.0018,10000,500,9.000
3,Codegen,0.0018,10000,1000,18.000
4,Codegen,0.0018,10000,10000,180.000


In [26]:
df[df['Model'] == 'Codegen']['cost'].mean()

np.float64(0.0018)

In [27]:
result = eval(code) 
result

np.float64(0.0018)